In [ ]:
import torch
import torch.nn as nn

class MLP(nn.Module):
    def __init__(self, input_size=784, hidden_size=589, num_classes=10): #hidden_size 100 -> 589
        super(MLP, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, num_classes)
        )

    def forward(self, x):
        """
        순전파 함수 // forward propagation
        x: 입력 텐서 (batch_size, 784)
        return: 출력 텐서 (batch_size, 10)
        """
        return self.layers(x)

# 모델 생성 및 구조 확인
model = MLP()
print("모델 구조:")
print(model)

# 파라미터 개수 계산
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"\n총 파라미터 수: {total_params:,}")
print(f"학습 가능한 파라미터 수: {trainable_params:,}")

# 각 레이어별 파라미터 수 확인
print("\n레이어별 파라미터:")
for name, param in model.named_parameters():
    print(f"{name}: {param.shape} ({param.numel():,} 개)")

# 하이퍼파라미터 설정
batch_size = 138        # 배치 크기 128 -> 138
test_batch_size = 1000  # 테스트 배치 크기 (메모리 효율을 위해 크게 설정)
learning_rate = 1e-3    # 학습률 (0.001)
nb_epochs = 4           # 에포크 수 3 -> 4

print("=== 하이퍼파라미터 ===")
print(f"배치 크기: {batch_size}")
print(f"테스트 배치 크기: {test_batch_size}")
print(f"학습률: {learning_rate}")
print(f"에포크 수: {nb_epochs}")

# 디바이스 설정 (GPU가 있으면 GPU 사용)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"\n사용 디바이스: {device}")
if device.type == "cuda":
    print(f"GPU 이름: {torch.cuda.get_device_name(0)}")
    print(f"GPU 메모리: {torch.cuda.get_device_properties(0).total_memory // 1024**3} GB")
